In [1]:
import pandas as pd

data = pd.read_csv("/Users/jeevakumar/Desktop/DS_Lab/data/project_data/UK_Farmer_Tweets.csv")

In [2]:
data.keys()

Index(['UniversalMessageId', 'SocialNetwork', 'SenderUserId',
       'SenderScreenName', 'SenderListedName', 'SenderProfileImgUrl',
       'SenderProfileLink', 'Sender Followers Count', 'SenderInfluencerScore',
       'SenderAge', 'SenderGender', 'OriginalAuthor',
       'OriginalAuthorProfileLink', 'Title', 'Message', 'MessageType',
       'CreatedTime', 'Language', 'LanguageCode', 'CountryCode',
       'MediaTypeList', 'Permalink', 'Domain', 'Retweets', 'Tweet Generator',
       'Favorites', 'ReceiverId', 'ReceiverScreenName', 'AssignedBy',
       'AssignedTo', 'Spam', 'Status', 'Intel Location', 'Priority',
       'Star Rating', 'Sentiment', 'Review Source',
       'Experience Score - Message level', 'ClientQueues', 'PartnerQueues',
       'ClientCustomProps', 'PartnerCustomProps', 'Custom Tags', 'Action Time',
       'Geo Target', 'Post Id', 'Associated Cases', 'Location', 'Country',
       'State', 'City', 'Latitude', 'Longitude', 'Sender Email',
       'Message Type', 'CampaignNa

In [3]:
senti_data = data[['Message', 'Sentiment']]

In [4]:
senti_data.describe()

,Message,Sentiment
count,10000,10000
unique,7062,3
top,RT @rihanna why aren’t we talking about this?!...,NEUTRAL
freq,83,5759


In [5]:
senti_data.describe()

,Message,Sentiment
count,10000,10000
unique,7062,3
top,RT @rihanna why aren’t we talking about this?!...,NEUTRAL
freq,83,5759


In [6]:
senti_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Message    10000 non-null  object
 1   Sentiment  10000 non-null  object
dtypes: object(2)
memory usage: 156.4+ KB


In [7]:
senti_data[senti_data.duplicated(["Message"],keep=False)]

,Message,Sentiment
5,RT @UK51NGH Sham of a democracy as the Indian ...,NEGATIVE
9,RT @UK51NGH Sham of a democracy as the Indian ...,NEGATIVE
10,RT @UK51NGH Sham of a democracy as the Indian ...,NEGATIVE
14,RT @timetospeakoutt @LizWebsterLD As @helenhal...,NEGATIVE
16,"RT @penninevale #Fishermen, #Farmers, #economy...",NEGATIVE
...,...,...
9993,RT @Jamie_Margolin I stand with the farmers in...,NEGATIVE
9994,RT @ClaudiaWebbe Solidarity to the Indian Farm...,NEUTRAL
9995,RT @GretaThunberg We stand in solidarity with ...,NEUTRAL
9997,RT @GretaThunberg We stand in solidarity with ...,NEUTRAL


In [8]:
#senti_data['Sentiment'] = senti_data['Sentiment'].astype('category')

In [9]:
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = ''.join((z for z in text if not z.isdigit()))
    # Tokenize text
    tokens = nltk.word_tokenize(text)
    # Remove stop words
    stop_words = set(stopwords.words("english"))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    return lemmatized_tokens

senti_data['tokenized_text'] = senti_data['Message'].apply(preprocess_text)

/var/folders/n5/ns746s3s12v7rsnsjqlplnbm0000gn/T/ipykernel_5342/4164890324.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  senti_data['tokenized_text'] = senti_data['Message'].apply(preprocess_text)


In [10]:
# from nltk.tokenize import TweetTokenizer
# tt = TweetTokenizer()
# senti_data.Message = senti_data.Message.apply(tt.tokenize)
senti_data['tokenized_text'] 

0       [panjabi, journalist, punyaab, great, reportin...
1       [rt, sukhwin, releasejagmeetsinghsham, democra...
2       [rt, ayalimanpreet, warmly, welcome, farmers, ...
3       [rt, harjapbhangal, sedition, meanwhile, peopl...
4       [rt, milffarmer, mmm, cum, show, offer, uncens...
                              ...                        
9995    [rt, gretathunberg, stand, solidarity, farmers...
9996                                   [thank, greta, 🙌🏽]
9997    [rt, gretathunberg, stand, solidarity, farmers...
9998    [following, awareness, raised, rihanna, anothe...
9999    [rt, khalsaaid, thank, gretathunberg, 🙏🏻, ’, g...
Name: tokenized_text, Length: 10000, dtype: object

In [11]:
senti_data.head()

,Message,Sentiment,tokenized_text
0,Panjabi Journalist of 2021 has to be @PunYaab ...,NEUTRAL,"[panjabi, journalist, punyaab, great, reportin..."
1,RT @Sukhwin19354152 #ReleaseJagmeetSinghSham o...,NEUTRAL,"[rt, sukhwin, releasejagmeetsinghsham, democra..."
2,RT @AyaliManpreet Warmly welcome the farmers a...,POSITIVE,"[rt, ayalimanpreet, warmly, welcome, farmers, ..."
3,RT @HarjapBhangal Sedition??! Meanwhile the pe...,NEGATIVE,"[rt, harjapbhangal, sedition, meanwhile, peopl..."
4,RT @milf_farmer_1 Mmm cum show me what you hav...,NEUTRAL,"[rt, milffarmer, mmm, cum, show, offer, uncens..."


In [12]:
#Baseline model
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Preprocess the data
senti_data['Sentiment'] = senti_data['Sentiment'].map({'POSITIVE': 2, 'NEUTRAL': 1, 'NEGATIVE': 0})

senti_data.dropna(inplace=True)
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(senti_data['Message'], senti_data['Sentiment'], test_size=0.2)

# Extract features from the tweets using bag-of-words representation
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Train the model using Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Test the model on the testing set and evaluate its performance
y_pred = clf.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))


/var/folders/n5/ns746s3s12v7rsnsjqlplnbm0000gn/T/ipykernel_5342/163504853.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  senti_data['Sentiment'] = senti_data['Sentiment'].map({'POSITIVE': 2, 'NEUTRAL': 1, 'NEGATIVE': 0})
/var/folders/n5/ns746s3s12v7rsnsjqlplnbm0000gn/T/ipykernel_5342/163504853.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  senti_data.dropna(inplace=True)


Accuracy: 0.801


In [47]:
import spacy
nlp = spacy.load('en_core_web_sm')

def find_aspects_tweet(doc):
    noun_adj_pairs = {}
    doc = nlp(doc)
    for chunk in doc.noun_chunks:
        adj = set()
        noun = ""
        for tok in chunk:
            if tok.pos_ == "NOUN":
                noun = tok.text
            if tok.pos_ == "ADJ":
                adj.add(tok.text.lower())
        if noun:
            noun_adj_pairs.update({noun.lower():adj})
    return noun_adj_pairs

senti_data['aspects'] = senti_data['Message'].apply(find_aspects_tweet)


/var/folders/n5/ns746s3s12v7rsnsjqlplnbm0000gn/T/ipykernel_5342/476584383.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  senti_data['aspects'] = senti_data['Message'].apply(find_aspects_tweet)


In [59]:
aspects = {}
# finding combining all aspects into a single dictionary
# removing aspects without adjectives ?
for aspect in senti_data['aspects']:
    for key, value in aspect.items():
        if key not in aspects and len(value) > 0:
            aspects[key] = set()
        elif len(value) > 0:
            aspects[key].update(value)
print (aspects)

{'reporting': {'ongoing', 'mandatory', 'different', 'shameful', 'original', 'truthful', '@bbcnews'}, 'state': {'authoritarian', 'populous', 'right', 'mighty', 'northern', 'indian', 'evil', 'sad', 'important'}, 'win': {'big', 'collective', 'easy', 'massive', 'huge'}, 'rallies': {'nazi'}, 'conferences': {'more'}, 'thing': {'other', 'only', 'favourite', 'insulting', 'right', 'worst', 'good', 'craziest', 'big', 'many', 'wise', 'important'}, 'year': {'75th', 'difficult', 'last', 'more', 'long', 'innocent', 'past', 'old', 'disastrous'}, 'support': {'ongoing', 'worldwide', 'such', 'incredible', 'pakistani', 'vocal', 'tactical', 'focused', 'huge', 'whopping', 'logical', 'global', 'full', 'fantastic', 'international', 'same', 'extra', 'financial', 'popular', 'free', 'long', 'more', 'friendly', 'restorative', 'great', 'big', 'dear', 'continuous', 'infinite', 'external', 'unwavering'}, 'video': {'@freejagginow', '▶', 'below', 'amazing', 'great', 'promotional', 'full', 'older', 'fake'}, 'governmen

In [78]:
# finding common words associated with different aspects

dep_aspects = {}
count = 0 
for key1, value1 in aspects.items():
    for key2, value2 in aspects.items():
        if key1 == key2:
            continue
        keys = sorted([key1, key2])
        combined_key = "{}__{}".format(keys[0], keys[1])
        if combined_key in dep_aspects:
            continue
        inter = value1.intersection(value2)
        if len(inter) > 0: # the sets are indepent with eachother
            dep_aspects[combined_key] = inter

# remove common words from aspect sets making them disjoint

for keys, values in dep_aspects.items():
    aspect1, aspect2 = keys.split("__")
    aspects[aspect1] = aspects[aspect1] - values
    aspects[aspect2] = aspects[aspect2] - values
print(aspects)

15414


{'reporting': {'mandatory', 'truthful', '@bbcnews'}, 'state': {'populous', 'evil', 'mighty'}, 'win': set(), 'rallies': set(), 'conferences': set(), 'thing': {'craziest', 'favourite', 'insulting'}, 'year': {'75th', 'disastrous'}, 'support': {'popular', 'restorative', 'vocal', 'tactical', 'whopping', 'logical', 'focused', 'infinite', 'external', 'unwavering'}, 'video': {'▶', 'promotional', '@freejagginow', 'older'}, 'government': {'coward', 'subsequent', 'exact', 'apathetic', 'federal'}, 'minorities': set(), 'radicals': set(), 'nation': {'secular', 'progressive'}, 'sense': {'palpable'}, 'moment': set(), 'attack': {'lakhimpur', 'unprovoked'}, 'deal': {'crass', 'dreadful', 'awful', 'planned', 'reciprocal'}, 'वाले': {'पहुंचने'}, 'exercise': set(), 'teacher': set(), 'gift': set(), 'storms': set(), 'propaganda': {'pseudo', 'fatf'}, 'constituency': set(), 'food': {'safer', '@sikhpa', 'industrialised', 'plentiful'}, 'air': {'cleaner'}, 'part': {'rajasthan', 'integral'}, 'community': {'ub1ub2', 

{'reporting_support': {'ongoing'}, 'gift_reporting': {'original'}, 'issues_reporting': {'ongoing'}, 'parties_reporting': {'different'}, 'protest_reporting': {'ongoing'}, 'protests_reporting': {'ongoing'}, 'problems_reporting': {'ongoing'}, '#_reporting': {'ongoing'}, 'movement_reporting': {'ongoing'}, 'reporting_violations': {'ongoing'}, 'efforts_reporting': {'ongoing'}, 'conditions_reporting': {'different'}, 'people_reporting': {'different'}, 'reporting_visas': {'different'}, 'reporting_words': {'original'}, 'reporting_stand': {'ongoing'}, 'reporting_version': {'original'}, 'challenges_reporting': {'ongoing'}, 'abuses_reporting': {'ongoing'}, 'choices_reporting': {'different'}, 'act_reporting': {'shameful'}, 'demonstrations_reporting': {'ongoing'}, 'platforms_reporting': {'different'}, 'parts_reporting': {'different'}, 'off_reporting': {'ongoing'}, 'reporting_types': {'different'}, 'case_reporting': {'ongoing'}, 'reporting_sites': {'different'}, 'disobedience_reporting': {'ongoing'}, 